In [1]:
import pandas as pd
import numpy as np
from mega import get_peaks, numf
from matplotlib import pyplot as plt

np.random.seed(42)

In [2]:
df = pd.read_csv('cases.csv')
df.head()

,dates,cases
0,1/22/20,0.857143
1,1/23/20,0.714286
2,1/24/20,0.428571
3,1/25/20,0.571429
4,1/26/20,0.571429


In [3]:
M = df['cases'].to_numpy().reshape(-1, 1)

In [4]:
r = 10
m, n = M.shape
W0 = np.random.rand(m, r)
H0 = np.random.rand(r, n)

In [5]:
print(np.linalg.norm(M - W0 @ H0, 'fro') / np.linalg.norm(M, 'fro'))

0.9998617304687226


In [ ]:
pvals = get_peaks(M, nrad=1)
W, H  = numf(M, W0, H0, iters=5, pvals=pvals)

In [ ]:
plt.plot(W)